# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are determined by the number of MAL recommendations between two series (see  https://myanimelist.net/recommendations.php?s=recentrecs&t=anime)

In [1]:
name = "RelatedSeries";

In [2]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

In [3]:
using DataFrames

## Compute similarity matrices

In [4]:
function get_similarity_matrix(relationships)
    file = "../../data/processed_data/related_series.csv"
    df = DataFrame(CSV.File(file))
    df = filter(x -> x.relationship ∈ relationships, df)
    num_items = maximum(get_split("training").item)
    sparse(
        df.source .+ 1,
        df.target .+ 1,
        fill(1.0, length(df.source)),
        num_items,
        num_items,
    )
end;

In [5]:
function all_pairs_shortest_paths(S)
    # uses the optimization that all edges are weight 1
    dists = fill(Inf, size(S)...)
    for v = 1:size(S)[1]
        dists[v, v] = 0
    end
    T = I(size(S)[1])

    @showprogress for epoch = 1:size(S)[1]
        T = T * S
        change = false
        for (i, j, v) in zip(findnz(T)...)
            if dists[i, j] > epoch
                dists[i, j] = epoch
                change = true
            end
        end
        if !change
            break
        end
    end
    dists
end

all_pairs_shortest_paths (generic function with 1 method)

In [6]:
function get_similarity_matrix_outdir(relationships)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(relationships))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for relationships $relationships"
    S = get_similarity_matrix(relationships)
    S = collect(1 ./ all_pairs_shortest_paths(S))
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [7]:
all_relations = [
    "side_story",
    "summary",
    "parent_story",
    "sequel",
    "prequel",
    "character",
    "alternative_version",
    "other",
    "spin_off",
    "alternative_setting",
    "full_story",
];

strict_relations = ["side_story", "parent_story", "sequel", "prequel"];

In [8]:
@memoize function max_neighborhood_size(relation)
    # returns the size of the largest nonzero neighborhood
    S = read_params(get_similarity_matrix_outdir(relation))["S"]
    S = 1 ./ S
    maxK = 0
    for j = 1:size(S)[2]
        K = sum((S[:, j] .!= 0) .* (S[:, j] .!= Inf))
        if K > maxK
            maxK = K
        end
    end
    neighborhood_size = Int(round(maxK))
    @debug "using neighborhood size $neighborhood_size"
    neighborhood_size
end;

In [9]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_alphas = [
    ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
    ["ItemCFEmbed.64", "ItemCFResid.256"]
]
params = [
    cf_params(
        name = "ItemCFRelated.$(hash(relation))",
        training_residuals = [["UserItemBiases"]; item_alphas],
        validation_residuals = [["UserItemBiases"]; item_alphas],
        neighborhood_type = "abs",
        S = get_similarity_matrix_outdir(relation),
        K = max_neighborhood_size(relation),
        λ = [1.0, 1.0, 0.0],
    ) for relation ∈ [strict_relations, all_relations]
];

[ Debug: 20211231 19:55:44 using neighborhood size 96
[ Debug: 20211231 19:55:54 using neighborhood size 361


## Train models

In [ ]:
for param in params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:01:42 (25.86 ms/it)
[ Debug: 20211231 19:58:26 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.249530525258636,-9.729437615610603e-5,0.00038665610649252826,-0.0003133778329081025) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.04547449489864468,0.0022116378809532923,-0.041129454564840066,0.06180486236536572): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, 

Iter     Function value   Gradient norm 
     0     1.249531e+00     3.866561e-04
 * Current step size: 1.0
 * time: 0.021929025650024414
 * g(x): [-9.729437615610603e-5, 0.00038665610649252826, -0.0003133778329081025]
 * x: [1.0, 1.0, 0.0]


Progress: 100%|███████████████████████████| Time: 0:01:43 (26.13 ms/it)
[ Debug: 20211231 20:00:17 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2495302680887408,-9.731105854435592e-5,0.0003866280270613511,-0.00031337586455379885) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.0455099852794912,0.002215664949449093,-0.04113837551215484,0.06181795216933023): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ra

     1     1.249252e+00     2.533889e-04
 * Current step size: 1351.3090354661322
 * time: 819.098662853241
 * g(x): [-7.998929262896915e-5, 5.367044042051096e-5, -0.0002533889184286202]
 * x: [1.1314747695997867, 0.47750810967849155, 0.42347029712351475]


Progress: 100%|███████████████████████████| Time: 0:01:52 (28.34 ms/it)
[ Debug: 20211231 20:14:03 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2491410484671115,-2.759187742646532e-6,-0.0004758451748002306,-0.00011615508818777253) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(0.1433827688764446,-0.0020995768552544733,0.0919275568544314,0.06453884390166954): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, R

...

Progress: 100%|███████████████████████████| Time: 0:02:37 (39.71 ms/it)
[ Debug: 20211231 22:47:38 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.248498472746696,1.9195687677387426e-7,-4.119306340950981e-6,-9.139187299977313e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(171.29226881271788,35.36011390417582,-2.2900497127304176,0.062360299059550976): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratings

    24     1.248498e+00     9.528635e-06
 * Current step size: 3.959878393509913
 * time: 10542.65458393097
 * g(x): [2.512731025669479e-6, 9.52863472097374e-6, -5.177350496585115e-11]
 * x: [0.713088468683528, 0.6071433963801135, 3704.497354142687]


Progress: 100%|███████████████████████████| Time: 0:03:01 (45.57 ms/it)
[ Debug: 20211231 22:57:25 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2484983856494571,9.79285446141772e-7,3.961438007172601e-6,-3.884983994613983e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(262.9023667921277,54.16678304236958,-5.2065808226277905,0.062370990549555846): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDat

    25     1.248498e+00     6.195247e-06
 * Current step size: 2.8335722561025314
 * time: 11030.968340873718
 * g(x): [-1.779294767558379e-6, -6.195247261480316e-6, -2.4657240677324826e-11]
 * x: [0.7083970843895591, 0.6019262606328231, 5139.906879597439]


Progress: 100%|███████████████████████████| Time: 0:02:29 (37.52 ms/it)
[ Debug: 20211231 23:04:52 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2484983387798936,-6.586444783287051e-7,-2.7889406368809055e-6,-1.9175986024808547e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(366.3823217324565,76.03670351255775,-5.337093427961103,0.06235517539779425): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratings